In [1]:
from bs4 import BeautifulSoup
import urllib
from urllib.request import urlopen
import re
import pandas as pd
#import numpy as np
import requests
from requests import get

# Pulling all links off of the search pages (up to 2000) and putting them in a dataframe to scrape.

In [2]:
url_template="http://www.indeed.com/jobs?q=%22Data+Scientist%22&l=USA&start={}"
max_results=2000
Linkdf=[]

for start in range(0, max_results, 10):
    url=url_template.format(start)
    html=requests.get(url)
    soup=BeautifulSoup(html.content,'html.parser', from_encoding="utf-8")
    
    #for each in soup.find_all(a_="href"):
    page_links=soup.find_all('a',{'href':re.compile("/rc/")})
    for items in page_links:
        Linkdf.append(items['href'])   

In [3]:
#Sanity Check
len(Linkdf)
#print(Linkdf)

1743

In [8]:
#This code allows the code to display the full website instead of truncating
pd.set_option('display.max_colwidth',-1)

#Moving it to a data frame
data = {'links':Linkdf}
df = pd.DataFrame(data, columns=['links'])

#append indeed.com to the front of each
df['Web'] = 'https://www.indeed.com'
df['URL'] = df.Web.str.cat(df.links)

#pull out just a list of the websites.
websites=list(df['URL'])

#Sanity Check
len(websites)
#print(websites)

1743

# Looping through websites....

In [6]:
#Loop

Title=[]
Descriptions=[]
Location=[]

for url in websites:
    response=get(url)
    soup=BeautifulSoup(response.text,'html.parser')
    
    try:
        description_containers= soup.find(class_='jobsearch-jobDescriptionText')
    except:
        description_containers='None Found'
    
    try:
        title_containers=soup.find('h3')
    except:
        title_containers='None Found'
        
    try:
        location_containers=soup.find('',{'class':'jobsearch-CompanyInfoWithoutHeaderImage'}).find_all('div')[-1]
    except:
        location_containers='None Found'
    
    job_descriptions=str(description_containers)
    job_title=str(title_containers.text)
    locations=str(location_containers.text)
    
    Descriptions.append(job_descriptions)
    Title.append(job_title)
    Location.append(locations)

In [9]:
#THIS GIVES US WHAT WE NEED BUT PUTS IT ALL IN THE SAME COLUMN
Search_Values = ['Qualifications', 'Skills', 'Responsibilities', 'Education', 'Requirement']
i=0

Qualifications = []

for n in range(0, len(Descriptions)):
    for values in Search_Values :
        soup=BeautifulSoup(Descriptions[n],'lxml')

        Qualifications_tag = soup.find('b', text=re.compile(values))
        try:
            ul_tag = Qualifications_tag.find_next('ul')
            Qualification=ul_tag.text
            Qualifications.append({i,Qualification})
        except AttributeError: 
            Qualification="None Found"
            Qualifications.append({i,Qualification})
    i=i+1

In [10]:
Qualifications

[{0, 'None Found'},
 {0, 'None Found'},
 {'\nYou will implement and validate predictive models as well as create and maintain statistical models with a focus on big data.\nYou will be exposed to and incorporate a variety of statistical and machine learning techniques such as logistic regression, experimental design, generalized linear models, mixed modeling, CHAID/decision trees, neural networks and ensemble models.\nYou’ll communicate with internal and external clients to understand business needs and provide analytical solutions.\nYou will use leading edge tools such as COGNOS, Watson Studio and Watson Machine Learning.\nYou’ll work in an Agile, collaborative environment, partnering with other scientists, engineers, and database administrators of all backgrounds and disciplines to bring analytical rigor and statistical methods to the challenges of predicting behaviors.',
  0},
 {0, 'None Found'},
 {0, 'None Found'},
 {1,
  'Bachelor’s or Master’s degree in a quantitative field\nKnowl

In [11]:
df_title=pd.DataFrame({'Job_Title':Title})
df_title

,Job_Title
0,Entry Level Data Scientist
1,Jr. Data Scientist
2,Entry Level - Associate Data Scientist
3,"Data Scientist, Analytics (University Grad)"
4,Data Scientist
5,Full Time Opportunities for Students or Recent Graduates: Data & Applied Sciences
6,"Data Scientist, Analytics, Intern"
7,Data Scientist
8,"Internship Data Scientist, Product Analytics, Analytics"
9,Data Scientist


In [12]:
df_locations=pd.DataFrame({'Location':Location})
df_locations

,Location
0,United States
1,"New York, NY 10036"
2,United States
3,"New York, NY 10017"
4,"Englewood, CO 80113"
5,"Redmond, WA"
6,"New York, NY 10017"
7,"New York, NY"
8,"New York, NY 10011"
9,"New York, NY"


In [13]:
#Full descriptions in dataframe
df_descriptions = pd.DataFrame({'Full_Description': Descriptions})
df_descriptions

Full_Description
0     <div class="jobsearch-jobDescriptionText" id="jobDescriptionText"><div><b>Introduction</b><br/>\nAt IBM we have an amazing opportunity to transform the world with cognitive technology. By using the vast amounts of information available today to identify new patterns and make new discoveries, we are helping cities become smarter, hospitals transform patient care, financial institutions minimize risk, and pharmaceuticals find cures for rare diseases.\n<br/>\n<br/>\nData scientists work with enterprise leaders and key decision makers to solve business problems by preparing, analyzing, and understanding data to deliver insight, predict emerging trends, and provide recommendations to optimize results. Data scientists use a variety of data (structured, unstructured, IoT streaming), analytics, AI tools, and programming languages often using a cloud infrastructure to handle the volume and veracity of data streams.<br/>\n<br/>\nArmed with data, modeling expertise, and analytic results, the data scientist communicates conclusions and recommendations to stakeholders in an organization's leadership structure. Business acumen is an important skill for data scientists to effectively communicate their findings to business leaders, data scientists need strong consulting, communication, visualization, and storytelling skills.<br/>\n<br/>\n<b>Your Role and Responsibilities</b><br/>\n<b>START DATES FOR THIS POSITION ARE IN 2020</b><br/>\n<br/>\nData Scientists are in demand across IBM's growth areas. If hired, you will be matched to a team based on business demand, location and fit. Join the forward-thinking teams at IBM solving some of the world’s most complex problems – there is no better place to grow your career!<br/>\n<br/>\nWhat You’ll Do as an Entry-Level Data Scientist:<ul><li>\nYou will implement and validate predictive models as well as create and maintain statistical models with a focus on big data.</li><li>\nYou will be exposed to and incorporate a variety of statistical and machine learning techniques such as logistic regression, experimental design, generalized linear models, mixed modeling, CHAID/decision trees, neural networks and ensemble models.</li><li>\nYou’ll communicate with internal and external clients to understand business needs and provide analytical solutions.</li><li>\nYou will use leading edge tools such as COGNOS, Watson Studio and Watson Machine Learning.</li><li>\nYou’ll work in an Agile, collaborative environment, partnering with other scientists, engineers, and database administrators of all backgrounds and disciplines to bring analytical rigor and statistical methods to the challenges of predicting behaviors.</li></ul><br/>\nWho You Are:<ul><li>\nYou are great at solving problems, debugging, troubleshooting, and designing &amp; implementing solutions to complex technical issues.</li><li>\nYou thrive on teamwork and have excellent verbal and written communication skills.</li><li>\nYou have strong technical and analytical abilities, a knack for driving impact and growth, and some experience with programming/scripting in a language such as Java or Python.</li><li>\nYou have a basic understanding of statistical programming in a language such as R, SAS, or Python.</li><li>\nYou have an interest in, understanding of, or experience with Design Thinking and Agile Development Methodologies</li></ul><br/>\n<b>\nRequired Professional and Technical Expertise</b><br/>\n<ul><li>Basic understanding of statistical programming in a language such as R, SAS, or Python.</li><li>\nExperience with programming/scripting in a language such as Java or Python.</li><li>\nKnowledge of statistical concepts such as regression, time series, mixed model, Bayesian, clustering, etc., to analyze data and provide insights.</li></ul><br/>\n<b>\nPreferred Professional and Technical Expertise</b><br/>\n<ul><li>Advanced knowledge of statistical concepts such as regression, time series, mixed model, Bayesian methods, clustering

In [14]:
FullDataSet = pd.concat([df_title,df_locations, df_descriptions], axis=1)
FullDataSet

Job_Title                Location  \
0     Entry Level Data Scientist                   United States            
1     Jr. Data Scientist                           New York, NY 10036       
2     Entry Level - Associate Data Scientist       United States            
3     Data Scientist, Analytics (University Grad)  New York, NY 10017       
4     Data Scientist                               Englewood, CO 80113      
...              ...                                               ...      
1738  Data Scientist                               New York, NY             
1739  Data Scientist                               San Francisco, CA        
1740  Dealer Network Analytics Data Scientist      Auburn Hills, MI 48326   
1741  Data Scientist                               Orange, CA               
1742  Data Scientist                               Kansas City, MO 64108    

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           